### rackcli sequence - h4x VCV Rack sequencing patches

Here's a reasonable sequencing workflow for scenarios where you 1) want to be able to reason about non-trivial sequences almost instantly, 2) use various forms of sequence notation (tablature, notes, etc.), 3) try sequences on different plugin modules, and 4) change/reprogram sequencers as quickly as we can conceive them.  Of course there's also series of companion 
[videos](https://www.youtube.com/channel/UCv-mq6lyycCbvbQiZclik7Q)!

The last is critical for me as I "try" to jam with my guitar playing friends who can tweak their rifs in real time and don't understand why it takes me so log to tweak my sequencers!

Here's an initial attempt at programming a CV sequencer (I'll hit on trigger/gate sequecers in a bit) with two sequences of length four. I'll express the sequence in a very simple notation as follows, but they've been programmed interactively in a patch named `_seq.vcv` with a [PS-16](https://github.com/MarcBoule/ImpromptuModular#phrase-seq-16):

```
C C# D D#
G# A A# B
```

Our goal it to replace it with's "Twinkle Twinkle Little Star" (TTLS):

```
C C G G A A G "twinkle twinkle little star"
F F E E D D C "how I wonder what you are"
G G F F E E D "up above the world so high"
G G F F E E D "like a diamond in the sky"
C C G G A A G "twinkle twinkle little star"
F F E E D D C "how I wonder what you are"
```

You'll notice some repeats, so here's a more compact version -- we'll implement this in a later coversation:

```
C C G G A A G "twinkle twinkle little star"
F F E E D D C "how I wonder what you are"
G G F F E E D "up above the world so high"
2:3 1 "like a diamond in the sky"
0:2 2
```

Notice that if you have lyrics changing on sequence repititions, multiple quoted strings could be added on those lines. Of course, adding lyrics are for documentation only, as typical squencers can't sing along ... yet. ;-)

Wanted to throw in comment on the range and potential of this h4x -- what else could we do?

1. alternative notations (e.g. guitar tabs, percussion tabs, midi files, etc.)
1. using a different rack sequencer, being able to a/b test several from the same notation
1. support gate/trigger sequencers, not just cv sequencers
1. other "types" of modules, like the various chord generators
1. higher-order patching with support for supporting modules and their settings (e.g. add additional clocks, sequential switches, chaos, etc.)

Basically, any module that has superpowers and has a bit of a programming curve is a great candidate for this kind of h4x!

In [1]:
from pprint import pprint as pp
from typing import List, Dict
import json

In [2]:
rackdir = '/Users/dirkleas/Desktop/h4x/_queue_/vcv/Rack'
patch = json.load(open(rackdir + '/_seq.vcv')) # two sequeces of length 4
# pp(patch['modules'][0])

Start with some basic CV/note support.

In [3]:
note_offset = 1.0 / 12.0
note_multiple = {'c':0, 'c#':1, 'db':1, 'd':2, 'd#':3, 'eb':3, 'e':4, 'f':5, 
                   'f#':6, 'gb':6, 'g':7, 'g#':8, 'ab':8, 'a':9, 'a#bb':10, 'b':11}
notes = ['c', 'c#', 'd', 'd#', 'e', 'f', 'f#', 'g', 'g#', 'a', 'a#', 'b']

def toCV(note: str) -> float:
    'covert note to cv'
    note = note.strip().lower()
    octave = 0
    if len(note) > 1 and note[1].isdigit(): # todo: add range check
        octave = - (4 - int(note[1]))
        note = note[:1] + note[(2):]
    try: return(note_multiple[note] * note_offset + octave)
    except: return None

def toNote(cv: float, sharpy: bool=True) -> str: # todo: add non-sharpy
    note, octave = notes[round((cv - int(cv)) / note_offset)], str(4 + int(cv))
    octave = octave if not octave == '4' else ''
    return(note if not octave else note[0] + octave + note[1:])

# n1 = ['c', 'c#', 'db', 'd', 'c3', 'c5#']
# cv = list(map(toCV, n1))
# n2 = list(map(toNote, cv))
# print(n1, cv, n2)

Here's TTLS from above codified.

In [4]:
ttls = '''
C C G G A A G "twinkle twinkle little star"
F F E E D D C "how I wonder what you are"
G G F F E E D "up above the world so high"
G G F F E E D "like a diamond in the sky"
C C G G A A G "twinkle twinkle little star"
F F E E D D C "how I wonder what you are"
'''

def parse_notation(notation: str) -> (List[List[str]], List[List[float]]) :
    'convert notation (C C# "lyrics") to note and cv data'
    notes = [s.split('"')[0].strip().split() for s in ttls.split('\n') if s]
    return(notes, [[toCV(n) for n in s] for s in notes])
           
print('notation:', ttls)
ttls, ttls_cv = parse_notation(ttls)
print('parse/remove lyrics:'); pp(ttls)
print('\nttls cv:\n', ttls_cv)

notation: 
C C G G A A G "twinkle twinkle little star"
F F E E D D C "how I wonder what you are"
G G F F E E D "up above the world so high"
G G F F E E D "like a diamond in the sky"
C C G G A A G "twinkle twinkle little star"
F F E E D D C "how I wonder what you are"

parse/remove lyrics:
[['C', 'C', 'G', 'G', 'A', 'A', 'G'],
 ['F', 'F', 'E', 'E', 'D', 'D', 'C'],
 ['G', 'G', 'F', 'F', 'E', 'E', 'D'],
 ['G', 'G', 'F', 'F', 'E', 'E', 'D'],
 ['C', 'C', 'G', 'G', 'A', 'A', 'G'],
 ['F', 'F', 'E', 'E', 'D', 'D', 'C']]

ttls cv:
 [[0.0, 0.0, 0.5833333333333333, 0.5833333333333333, 0.75, 0.75, 0.5833333333333333], [0.41666666666666663, 0.41666666666666663, 0.3333333333333333, 0.3333333333333333, 0.16666666666666666, 0.16666666666666666, 0.0], [0.5833333333333333, 0.5833333333333333, 0.41666666666666663, 0.41666666666666663, 0.3333333333333333, 0.3333333333333333, 0.16666666666666666], [0.5833333333333333, 0.5833333333333333, 0.41666666666666663, 0.41666666666666663, 0.3333333333333333, 0.33333

Let's drill down into the CV note data. As usual, we'll assume C4 (middle C) is octave offset zero. , and the offset goes negative or positive to go either direction. Notationally, C = C4, C# = C4#, etc., but for convenience, the octave can be left off in the 4th octave. It's important to know sequence lengths!

In [5]:
patch = json.load(open(rackdir + '/_seq.vcv')) # original two sequeces of length 4
cv = patch['modules'][0]['data']['cv']
print('lengths:', patch['modules'][0]['data']['steps'], patch['modules'][0]['data']['phrases'], '\n', cv)

lengths: 4 4 
 [0.0, 0.0833333358, 0.166666672, 0.25, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.666666687, 0.75, 0.833333373, 0.916666687, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0

Now let's h4x the patch with some TTLS -- saved as new patch (for comparison), so open `seq2.vcv` and enjoy! You'll have to manually iterate through the sequences, but that easily be added it the next round of h4x!

In [6]:
def patch_sequencer(patch: Dict, cv: List[float], verbose: bool=False) -> None:
    'patch sequencer by updating patch with notational cv'
    patch['modules'][0]['data']['cv'] = [0.0] * 256 # clear sequences
    patch['modules'][0]['data']['phrase'] = [0] * 16 # clear phrases
    for i, s in enumerate(cv): patch['modules'][0]['data']['cv'][i*16:i*16+len(cv[0])] = s # sequences
    patch['modules'][0]['data']['phrase'][0:len(cv)] = range(0, len(cv))
    patch['modules'][0]['data']['steps'] = len(cv[0]) # steps
    patch['modules'][0]['data']['phrases'] = len(cv) # phrases
    json.dump(patch, open(rackdir + '/_seq2.vcv', 'w'), indent=2, ensure_ascii=False) # ttls ftw!
    if verbose:
        print('lengths:', patch['modules'][0]['data']['steps'], patch['modules'][0]['data']['phrases'], '\n', 
              patch['modules'][0]['data']['cv'], patch['modules'][0]['data']['phrase'])

patch_sequencer(patch, ttls_cv, verbose=True)

lengths: 7 6 
 [0.0, 0.0, 0.5833333333333333, 0.5833333333333333, 0.75, 0.75, 0.5833333333333333, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.41666666666666663, 0.41666666666666663, 0.3333333333333333, 0.3333333333333333, 0.16666666666666666, 0.16666666666666666, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.5833333333333333, 0.5833333333333333, 0.41666666666666663, 0.41666666666666663, 0.3333333333333333, 0.3333333333333333, 0.16666666666666666, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.5833333333333333, 0.5833333333333333, 0.41666666666666663, 0.41666666666666663, 0.3333333333333333, 0.3333333333333333, 0.16666666666666666, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.5833333333333333, 0.5833333333333333, 0.75, 0.75, 0.5833333333333333, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.41666666666666663, 0.41666666666666663, 0.3333333333333333, 0.3333333333333333, 0.16666666666666666, 0.16666666666666666, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0

How about a different sequencer?

In [7]:
# as seq16, paramId:3=>steps, rows 8..23,24..39,40..55 0..10, everything in params
patch = json.load(open(rackdir + '/eraseme.vcv'))
params = patch['modules'][0]['params']

def param(x, val):
    'set paramId x to value, return success truth'
    try: [p for p in params if p['paramId'] == x][0]['value'] = val; return True
    except: return False

tls_cv = ttls_cv[:3] # only process first three sequences
for p in range(8,56): param(p, 0.0) # clear cv
for i, s in enumerate(ttls_cv): #patch['modules'][0]['data']['cv'][i*16:i*16+len(ttls_cv[0])] = s # sequences
    for j, cv in enumerate(s):
        param(i*16+8+j, cv)
param(3, len(ttls_cv[0])) # update steps
json.dump(patch, open(rackdir + '/eraseme2.vcv', 'w'), indent=2, ensure_ascii=False) # ttls ftw!
print(patch['modules'][0])

{'plugin': 'AS', 'version': '0.6.7', 'model': 'SEQ16', 'pos': [12, 0], 'width': 660, 'params': [{'paramId': 0, 'value': 2.0}, {'paramId': 1, 'value': 0.0}, {'paramId': 2, 'value': 0.0}, {'paramId': 4, 'value': 0.0}, {'paramId': 5, 'value': 0.0}, {'paramId': 6, 'value': 0.0}, {'paramId': 7, 'value': 0.0}, {'paramId': 3, 'value': 7}, {'paramId': 8, 'value': 0.0}, {'paramId': 24, 'value': 0.41666666666666663}, {'paramId': 40, 'value': 0.5833333333333333}, {'paramId': 56, 'value': 0.5833333333333333}, {'paramId': 9, 'value': 0.0}, {'paramId': 25, 'value': 0.41666666666666663}, {'paramId': 41, 'value': 0.5833333333333333}, {'paramId': 57, 'value': 0.5833333333333333}, {'paramId': 10, 'value': 0.5833333333333333}, {'paramId': 26, 'value': 0.3333333333333333}, {'paramId': 42, 'value': 0.41666666666666663}, {'paramId': 58, 'value': 0.41666666666666663}, {'paramId': 11, 'value': 0.5833333333333333}, {'paramId': 27, 'value': 0.3333333333333333}, {'paramId': 43, 'value': 0.41666666666666663}, {'p